In [1]:

!pip install lightgbm
!pip install scikit-learn

import sys
!{sys.executable} -m pip install --upgrade jupyter notebook ipykernel

!pip install xgboost

In [2]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#한글 폰트 설정 - title 한글 깨짐
plt.rc('font', family = 'NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

import lightgbm
from sklearn.metrics import r2_score

from tqdm import tqdm

import pickle
import warnings;warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
load_dt = pd.read_csv('/data/ephemeral/home/train.csv')
load_dt_test = pd.read_csv('/data/ephemeral/home/test.csv')

load_dt['is_test'] = 0
load_dt_test['is_test'] = 1
load_dt_test['target'] = 0

select_columns = ['시군구', '아파트명','전용면적(㎡)', '계약년월', '층', '건축년도', 'target', 'is_test'] 
temp = pd.concat([load_dt, load_dt_test])
concat_data = temp[select_columns]


In [4]:
#시군구, 도로명분리 및 전용면적 m2 제거

# 시군구를 시, 구, 동으로 분리

concat_data = concat_data.rename(columns={'전용면적(㎡)':'전용면적'})

print(concat_data.head(10))

             시군구    아파트명   전용면적    계약년월  층  건축년도  target  is_test
0  서울특별시 강남구 개포동  개포6차우성  79.97  201712  3  1987  124000        0
1  서울특별시 강남구 개포동  개포6차우성  79.97  201712  4  1987  123500        0
2  서울특별시 강남구 개포동  개포6차우성  54.98  201712  5  1987   91500        0
3  서울특별시 강남구 개포동  개포6차우성  79.97  201801  4  1987  130000        0
4  서울특별시 강남구 개포동  개포6차우성  79.97  201801  2  1987  117000        0
5  서울특별시 강남구 개포동  개포6차우성  79.97  201801  1  1987  130000        0
6  서울특별시 강남구 개포동  개포6차우성  79.97  201803  2  1987  139500        0
7  서울특별시 강남구 개포동  개포6차우성  54.98  201804  5  1987  107500        0
8  서울특별시 강남구 개포동  개포6차우성  79.97  201806  3  1987  145000        0
9  서울특별시 강남구 개포동  개포6차우성  54.98  201807  3  1987  112000        0


In [5]:
print(concat_data.isnull().sum())

시군구           0
아파트명       2136
전용면적          0
계약년월          0
층             0
건축년도          0
target        0
is_test       0
dtype: int64


In [6]:
# 결측치가 없는 아파트명과 있는 아파트명 데이터 분리
apartment_name = concat_data[concat_data['아파트명'].notna()]
no_apartment_name = concat_data[concat_data['아파트명'].isna()]

# 모델에 사용할 특성 선택
features = ['시군구', '전용면적', '건축년도']

# Label Encoding 설정
le_apt = LabelEncoder()
le_sgg = LabelEncoder()

# 아파트명 인코딩
apartment_name['아파트명_encoded'] = le_apt.fit_transform(apartment_name['아파트명'])

# 특성과 타겟 설정
X = apartment_name[features]
y = apartment_name['아파트명_encoded']

# 시군구 인코딩
le_sgg.fit(apartment_name['시군구'].astype(str))
X['시군구'] = le_sgg.transform(X['시군구'].astype(str))

# 결측값 대체를 위한 SimpleImputer 사용
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# 학습 및 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest 모델 설정 및 훈련
rf = RandomForestClassifier(n_estimators=50,
                            random_state=42, 
                            n_jobs=2, 
                            max_depth=13, 
                            min_samples_split=20, 
                            min_samples_leaf=10,
                            warm_start=False)
rf.fit(X_train, y_train)

# 검증 데이터 정확도 확인
val_accuracy = rf.score(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy:.4f}")

# 결측치가 있는 데이터에 대해 예측 준비
X_missing = no_apartment_name[features]

# 시군구 값이 인코더에 없는 경우 학습된 값의 첫 번째 값으로 대체
X_missing['시군구'] = X_missing['시군구'].apply(lambda x: x if x in le_sgg.classes_ else le_sgg.classes_[0])
X_missing['시군구'] = le_sgg.transform(X_missing['시군구'].astype(str))

# 결측값 채우기 및 예측
X_missing = pd.DataFrame(imputer.transform(X_missing), columns=X_missing.columns)
predicted_names_encoded = rf.predict(X_missing)
predicted_names = le_apt.inverse_transform(predicted_names_encoded)

# 예측된 아파트명으로 결측치 채우기
concat_data.loc[concat_data['아파트명'].isna(), '아파트명'] = predicted_names

# 남은 결측치 확인
print("남은 결측치 수:", concat_data['아파트명'].isnull().sum())

# 예측 결과 확인
print("\n예측된 아파트명 상위 10개:")
print(concat_data.loc[concat_data.index.isin(no_apartment_name.index), '아파트명'].value_counts().head(10))

print("\n전체 데이터셋 아파트명 상위 10개:")
print(concat_data['아파트명'].value_counts().head(10))

Validation Accuracy: 0.6588
남은 결측치 수: 0

예측된 아파트명 상위 10개:
구로두산       163
거평프리젠      151
한스빌        136
리센츠        130
대영빌라       101
중명하니빌       95
관악캠퍼스타워     84
방화5         57
성원          45
건영          45
Name: 아파트명, dtype: int64

전체 데이터셋 아파트명 상위 10개:
현대       16791
신동아      12960
한신        9139
두산        8601
주공2       7985
우성        7824
벽산        7330
삼성래미안     7185
대림        6349
극동        5697
Name: 아파트명, dtype: int64


In [7]:
concat_data.isnull().sum()  

시군구        0
아파트명       0
전용면적       0
계약년월       0
층          0
건축년도       0
target     0
is_test    0
dtype: int64

In [8]:
# #범주형 데이터, 연속형 데이터 라벨 
# continuous_columns = []
# categorical_columns = []

# for column in concat_data.columns:
#     if pd.api.types.is_numeric_dtype(concat_data[column]):
#         continuous_columns.append(column)
#     else:
#         categorical_columns.append(column)


# # 범주형 변수에 대한 보간
# concat_data[categorical_columns] = concat_data[categorical_columns].fillna('NULL')

# # 연속형 변수에 대한 보간 (선형 보간)
# concat_data[continuous_columns] = concat_data[continuous_columns].interpolate(method='linear', axis=0)

# label_encoders = {}
# # Implement Label Encoding
# for col in tqdm(categorical_columns):
#     lbl = LabelEncoder()

#     # Label-Encoding을 fit
#     lbl.fit( concat_data[col].astype(str) )
#     concat_data[col] = lbl.transform(concat_data[col].astype(str))
#     label_encoders[col] = lbl      


for column in concat_data.columns:
    if concat_data.dtypes[column] == 'object':
        concat_data[column] = concat_data[column].astype('category')

train_data = concat_data.query('is_test==0')
train_data.drop(columns='is_test', inplace=True)

temp_test_data =  concat_data.query('is_test==1')
except_target_select_columns = ['시군구', '아파트명', '층', '전용면적', '계약년월', '건축년도'] 
test_data = temp_test_data[except_target_select_columns] 

x = train_data[except_target_select_columns]
y = train_data['target']

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=10)



In [9]:
except_target_select_columns = ['시군구', '아파트명','전용면적', '계약년월', '층', '건축년도'] 

train_data = concat_data.query('is_test==0')
test_data =  concat_data.query('is_test==1')

x = train_data[except_target_select_columns]
y = train_data['target']

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=10)





In [10]:
print(test_data.columns)
print(train_data.columns)

Index(['시군구', '아파트명', '전용면적', '계약년월', '층', '건축년도', 'target', 'is_test'], dtype='object')
Index(['시군구', '아파트명', '전용면적', '계약년월', '층', '건축년도', 'target', 'is_test'], dtype='object')


In [11]:

model = lightgbm.LGBMRegressor(
        boosting_type='gbdt',      # 기본 부스팅 방식 (gbdt는 그레디언트 부스팅 결정 트리)
        objective='regression',    # 회귀 문제를 해결하기 위한 목적 함수 설정
        num_leaves=100,             # 리프 노드의 최대 개수
        learning_rate=0.1,         # 학습률 (작을수록 학습 속도가 느려짐, 과적합 방지)
        n_estimators=10000,         # 부스팅 반복 횟수 (트리 개수)
        max_depth=-1,              # 트리의 최대 깊이 (-1은 제한 없음)
        min_child_samples=20,      # 리프 노드의 최소 데이터 수 (과적합 방지)
        subsample=0.5,            # 각 트리 학습 시 사용할 샘플 비율 (0.8은 80%만 사용)
        colsample_bytree=0.5,      # 각 트리 학습 시 사용할 특징 비율 (0.8은 80%만 사용)
        reg_alpha=0.1,             # L1 정규화 (과적합 방지)
        reg_lambda=0.1,             # L2 정규화 (과적합 방지)
        max_bin = 2000
    )



In [12]:
model.fit(x_train, y_train)

y_train_pred = model.predict(x_train)
y_pred = model.predict(x_valid)

mse = mean_squared_error(y_train_pred, y_train)
rmse = mean_squared_error(y_train_pred, y_train, squared=False)
mae = mean_absolute_error(y_train_pred, y_train)
r2 = r2_score(y_train_pred, y_train)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R2: {r2}")

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001839 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7367
[LightGBM] [Info] Number of data points in the train set: 895057, number of used features: 6
[LightGBM] [Info] Start training from score 57971.762336


MSE: 9420863.835420586
RMSE: 3069.342573812931
MAE: 1868.7916888932143
R2: 0.9956083804261004


In [13]:
test_data.drop(columns=['target','is_test'], axis=1, inplace=True)
print(model.feature_name_)
print(test_data.columns)

['시군구', '아파트명', '전용면적', '계약년월', '층', '건축년도']
Index(['시군구', '아파트명', '전용면적', '계약년월', '층', '건축년도'], dtype='object')


In [14]:
mse = mean_squared_error(y_valid, y_pred)
rmse = mean_squared_error(y_valid, y_pred, squared=False)
mae = mean_absolute_error(y_valid, y_pred)
r2 = r2_score(y_valid, y_pred)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R2: {r2}")

MSE: 26454375.523401327
RMSE: 5143.3817205610285
MAE: 2579.0762007168687
R2: 0.9876095807774438


In [15]:
result = model.predict(test_data)


In [16]:

output = pd.DataFrame(result.astype(int), columns=["target"])
output.to_csv('output1.csv', index=False)